# Deliver your first site

### Get started
​Akamai​ helps deliver your sites and apps faster and more securely. Follow this workflow to deliver a basic site using Akamai.

> <br/> **Brand-new to Akamai?**
> <br/>Check out [Get to know ​Akamai](https://techdocs.akamai.com/platform-basics/docs/welcome)​ to understand how ​Akamai​ handles content delivery over the Internet.
> <br/><br/>

### Many ways to get the job done
Every website delivery scenario is unique, so ​Akamai​ offers a wide range of tools and delivery methods. 

Here are the tools you can select. Click each tool to see overview.
* [Control Center](https://techdocs.akamai.com/control-ctr/docs)
* [API](https://techdocs.akamai.com/property-mgr/reference/api)
* [​CLI](https://github.com/akamai/cli)

### What's inside here
In order to speed up your learning experience, we provide you with the following resources.
* Origin web server
* Control Center login credential
* API credential
* Workspace (What you are seeing)

> <br/> **Note:** All resources are valid for **2 hours** only. Please complete your learning within that time window.
> <br/><br/>

### What you can expect when finished
Your website will be accessible on the largest, globally distributed content delivery network (CDN). This brings your site physically closer to your customers, dramatically speeding up its delivery time.

### What you will do
Onboarding follows a phased approach. In this workflow, you will complete each phase using a tool you selects.

<img width="70%" src="img/onboarding-flow-v8.jpg"/>
<br/><br/><br/>

1. **Get access.** We provide you with temporary account information so that you can access your contract, group, user, and API credential.

2. **Get your certificate.** For requests between a client and the ​Akamai​ network, you'll first need to set up a certificate to secure that connection.

3. **Ready your origin server.** Use ​the origin web server we provides.

4. **Define your property.** Create a new property to configure how you want to deliver your content with rules and behaviors.

5. **Test and go live.** We recommend that you test your property and then you can push it to production to start delivering your site through the ​Akamai​ network.

> <br/> **Action:** Click the tool you want to use for this workflow.
> <br/><br/>

&nbsp;&nbsp;[Control Center]() | [CLI]() | [API]()

In [4]:
# set environment variables
ctr_id=$(akamai pm lg -f json | jq -r .[0].contractIds[0])
grp_id=$(akamai pm lg -f json | jq -r .[0].groupId)

# if you know produtId, export it directly.
# prd_id='prd_Fresca'
# if you don't know productId, run this. 
prd_id=$(akamai pm lp -c $ctr_id -f json | jq -r .[0].productId)

prefix=$(python3 src/createName.py)
p_name=$prefix'.akamai-lab.com'
eh_name=$p_name'.edgesuite.net'

c_secret=$(awk '/client_secret/ {print$3}' credential.txt | sort -u)
host=$(awk '/host/ {print$3}' credential.txt | sort -u)
a_token=$(awk '/access_token/ {print$3}' credential.txt | sort -u)
c_token=$(awk '/client_token/ {print$3}' credential.txt | sort -u)

echo 'Your property name:' $p_name
echo 'Your CP code name:' $prefix
echo 'Your hostname:' $p_name
echo 'Your edge hostname:' $eh_name

Your property name: 894710-charcoal-availability.akamai-lab.com
Your CP code name: 894710-charcoal-availability
Your hostname: 894710-charcoal-availability.akamai-lab.com
Your edge hostname: 894710-charcoal-availability.akamai-lab.com.edgesuite.net


In [7]:
akamai pm lp -c $ctr_id -f json

[
    {
        "productName": "Fresca",
        "productId": "prd_Fresca"
    }
]


Create your [CP code](https://techdocs.akamai.com/onboard/docs/set-up-identity-and-access-api#3-get-your-cp-codes)

In [5]:
akamai pm create-cpcode -c $ctr_id -g $grp_id -d $prd_id -n $p_name -f json

In [6]:
cp_id=$(akamai pm list-cpcodes -c $ctr_id -g $grp_id -f json | jq  -r --arg name $p_name  '.[] | select(.cpcodeName == $name) | .cpcodeId' | cut -c 5-)
echo $cp_id

1462526


Create a new property.

In [8]:
akamai property-manager new-property -c $ctr_id -g $grp_id -d $prd_id -p $p_name --insecure 

In [9]:
akamai pm lpr -c $ctr_id -g $grp_id

╒══════════════════╤═══════════════╤════════════╤══════════════════════════════╤═════════════╤══════════════╤════════════════════╤════════════════╤═════════════════╕
│"Account Id"      │"Contract Id"  │"Group Id"  │"Property Name"               │"Property Id"│"Asset Id"    │"Production Version"│"Latest Version"│"Staging Version"│
╞══════════════════╪═══════════════╪════════════╪══════════════════════════════╪═════════════╪══════════════╪════════════════════╪════════════════╪═════════════════╡
│"act_F-AC-4917414"│"ctr_V-41DUHPB"│"grp_232865"│"20230403123732.akamai-lab.com│"prp_907470" │"aid_11149225"│null                │1               │1                │
│                  │               │            │"                             │             │              │                    │                │                 │
├──────────────────┼───────────────┼────────────┼──────────────────────────────┼─────────────┼──────────────┼────────────────────┼────────────────┼─────────────────┤
│"ac

Update main.json and Augmented_insights.json

Edit **main.json** in *workspaces > lab2 > $p_name > config-snippets*.

Update "hostname": "origin-XXXX" to your origin hostname. For example, "origin--main--20230405030614--a.b.akamai-lab.com"

Change "forwardHostHeader": *"REQUEST_HOST_HEADER"* to "forwardHostHeader": *"ORIGIN_HOSTNAME"*

**screencast placeholder**

Edit **Augment_insights.json** in *workspaces > lab2 > test001 > config-snippets* directory.

Update "cpCode",
        "options":{}
        
to
        
          "options": {
            "value": {
              "id": $cp_id
            }
          }

main.json - origin hostname

Augmented_insights.json - cp code
{
    "name": "cpCode",
    "options": {}
}

{
                "name": "cpCode",
                "options": {
                    "value": {
                        "id": 1459812,
                        "name": ""
                    }
                }
            }

In [11]:
# create edge hostname, add property hostname
akamai pm save -p $p_name

╒════════════════════════╤════════╕
│"Action"                │"Result"│
╞════════════════════════╪════════╡
│"stored rule tree"      │"yes"   │
├────────────────────────┼────────┤
│"edge hostnames created"│"yes"   │
├────────────────────────┼────────┤
│"stored hostnames"      │"yes"   │
├────────────────────────┼────────┤
│"validation warnings"   │"yes"   │
├────────────────────────┼────────┤
│"validation errors"     │"no"    │
├────────────────────────┼────────┤
│"hostname warnings"     │"no"    │
├────────────────────────┼────────┤
│"hostname errors"       │"no"    │
└────────────────────────┴────────┘
There are validation warnings: 
 [
    {
        "type": "https://problems.luna.akamaiapis.net/papi/v0/validation/validation_message.general_warning",
        "errorLocation": {
            "template": "config-snippets/Augment_insights.json",
            "variables": [],
            "location": "children/2/criteria/0",
            "value": {
                "name": "requestType",
     

In [12]:
# Activate property. This can take a couple of minutes.
akamai pm activate -p $p_name -m 'first version' -n staging -w

Following activations are now pending:
╒══════════════════════════════╤═════════╤═══════════════╕
│"Property"                    │"Network"│"Activation Id"│
╞══════════════════════════════╪═════════╪═══════════════╡
│"894710-charcoal-availability.│"STAGING"│12957889       │
│akamai-lab.com"               │         │               │
└──────────────────────────────┴─────────┴───────────────┘
...activation status is PENDING...
...Waiting for active status...
...Checking activations...
...activation status is PENDING...
...Waiting for active status...
...Checking activations...
...activation status is PENDING...
...Waiting for active status...
...Checking activations...
185.202 seconds since command ran
Activation status report:
╒════════════════════════════════════════╤═════════╤═══════════════╤════════╕
│"Property"                              │"Network"│"Activation Id"│"Status"│
╞════════════════════════════════════════╪═════════╪═══════════════╪════════╡
│"894710-charcoal-availability.

Your site is akamaized on Akamai staging network. Visit [http://$p_name](http://894710-charcoal-availability.akamai-lab.com), which is akamaized on Akamai staging network.

screencast placeholder

Edit **Augment_insights.json** in *workspaces > lab2 > test001 > config-snippets* directory.

Update "cpCode",
        "options":{}
        
to
        
          "options": {
            "value": {
              "name": "test001",
            }
          }

screencast placeholder